In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow import keras

In [2]:
data = pd.read_csv('Data_folder/robo_coord_angle_dataset.csv')

In [3]:
data.isnull().any()

Unnamed: 0    False
x_coord       False
y_coord       False
z_coord       False
alpha         False
beta          False
gamma         False
dtype: bool

In [4]:
data.head(10)

,Unnamed: 0,x_coord,y_coord,z_coord,alpha,beta,gamma
0,0,12.524,-50.233,1432.786,104,-38,-90
1,1,7.796,-31.269,1432.445,104,-38,-88
2,2,3.074,-12.329,1431.422,104,-38,-86
3,3,-1.637,6.565,1429.718,104,-38,-84
4,4,-6.330,25.389,1427.336,104,-38,-82
5,5,-11.001,44.122,1424.278,104,-38,-80
6,6,-15.643,62.739,1420.549,104,-38,-78
7,7,-20.250,81.219,1416.152,104,-38,-76
8,8,-24.818,99.539,1411.093,104,-38,-74
9,9,-29.340,117.677,1405.378,104,-38,-72


In [5]:
X_df = data.iloc[:, 1:4]
Y_df = data.iloc[:, 4:]
X = X_df.values
Y = Y_df.values

In [6]:
#splitting the dataset into train, valid, test in the ratio  of 6:2:2
Xtrainval, Xtest, Ytrainval, Ytest = train_test_split(X, Y, test_size=0.2, random_state = 42)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrainval, Ytrainval, test_size=0.16, random_state = 42)

In [7]:
#scaling (standard) X data as per Xtrain
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xval = scaler.transform(Xval)
Xtest = scaler.transform(Xtest)

In [8]:
from keras import backend as K
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [9]:
#model
def model_development(neurons, layrs, gap) :
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal', input_shape=Xtrain.shape[1:]))
    if layrs <= 5:
        for i in range(layrs-1):
            model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal'))
    else:
        flag = 0
        for i in range(0,layrs,gap+1):
            if flag == 1:
                for j in range(gap):
                    model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal'))
            else:
                for j in range(gap-1):
                    model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal'))
                flag = 1
    model.add(keras.layers.Dense(3))
    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=0.001),metrics=[coeff_determination])
    return model

In [9]:
#callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint("IKmodel.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
lr_schedule = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)

In [11]:
#grid used in RandomSearchCV
grid = {
    'neurons' : [10, 30, 50],
    'layrs' : [5, 10, 20, 30],
    'gap' : [1, 2, 3]
}

In [12]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(model_development)
rnd_search_cv = RandomizedSearchCV(keras_reg, grid, n_iter=1, cv=3)

In [20]:
rnd_search_cv.fit(Xtrain, Ytrain, epochs=100, validation_data=(Xval, Yval),
callbacks=[checkpoint_cb, early_stopping, lr_schedule])

C:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
2513/2513 [==============================] - 10s 3ms/step - loss: 209.9378 - coeff_determination: 0.9602 - val_loss: 106.9130 - val_coeff_determination: 0.9799
Epoch 2/100
2513/2513 [==============================] - 8s 3ms/step - loss: 108.9700 - coeff_determination: 0.9793 - val_loss: 110.5785 - val_coeff_determination: 0.9792
Epoch 3/100
2513/2513 [==============================] - 8s 3ms/step - loss: 107.0384 - coeff_determination: 0.9797 - val_loss: 104.5958 - val_coeff_determination: 0.9803
Epoch 4/100
2513/2513 [==============================] - 8s 3ms/step - loss: 105.3981 - coeff_determination: 0.9800 - val_loss: 104.5338 - val_coeff_determination: 0.9803
Epoch 5/100
2513/2513 [==============================] - 8s 3ms/step - loss: 103.6589 - coeff_determination: 0.9803 - val_loss: 103.1988 - val_coeff_determination: 0.9806
Epoch 6/100
2513/2513 [==============================] - 9s 3ms/step - loss: 103.4126 - coeff_determination: 0.9804 - val_loss: 106.0846 - val_c

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000000010FDA2B0>,
                   n_iter=1,
                   param_distributions={'gap': [1, 2, 3],
                                        'layrs': [5, 10, 20, 30],
                                        'neurons': [10, 30, 50]})

In [22]:
rnd_search_cv.best_params_

{'neurons': 50, 'layrs': 10, 'gap': 3}

In [24]:
# In two fit iterations it can be seen that the best params are with lower values of layrs
# hence, some further analysis in lower layrs values is required
# another observation is the performance is better with larger values of gap or if no BatchNormalization layer
# is not present.

# So far the best r2 score on val data has been 98.19

In [10]:
#new model with no BatchNormalization and lower values of layrs
def model_development1(neurons, layrs) :
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal', input_shape=Xtrain.shape[1:]))
    for i in range(layrs-1):
        model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal'))
    model.add(keras.layers.Dense(3))
    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=0.001),metrics=[coeff_determination])
    return model

In [11]:
#grid1 used in RandomSearchCV
grid1 = {
    'neurons' : [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60],
    'layrs' : [4, 6, 8, 10, 12]
}

In [13]:
keras_reg1 = keras.wrappers.scikit_learn.KerasRegressor(model_development1)
rnd_search_cv1 = RandomizedSearchCV(keras_reg1, grid1, n_iter=1, cv=3)

In [15]:
rnd_search_cv1.fit(Xtrain, Ytrain, epochs=100, validation_data=(Xval, Yval),
callbacks=[checkpoint_cb, early_stopping, lr_schedule])

C:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
2513/2513 [==============================] - 9s 3ms/step - loss: 239.1693 - coeff_determination: 0.9549 - val_loss: 107.9350 - val_coeff_determination: 0.9797
Epoch 2/100
2513/2513 [==============================] - 8s 3ms/step - loss: 107.3735 - coeff_determination: 0.9796 - val_loss: 110.7580 - val_coeff_determination: 0.9792
Epoch 3/100
2513/2513 [==============================] - 7s 3ms/step - loss: 105.4122 - coeff_determination: 0.9800 - val_loss: 105.4163 - val_coeff_determination: 0.9802
Epoch 4/100
2513/2513 [==============================] - 8s 3ms/step - loss: 103.4311 - coeff_determination: 0.9803 - val_loss: 106.9895 - val_coeff_determination: 0.9799
Epoch 5/100
2513/2513 [==============================] - 7s 3ms/step - loss: 102.4417 - coeff_determination: 0.9806 - val_loss: 102.5596 - val_coeff_determination: 0.9807
Epoch 6/100
2513/2513 [==============================] - 8s 3ms/step - loss: 101.7790 - coeff_determination: 0.9807 - val_loss: 105.0596 - val_co

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000000005921130>,
                   n_iter=1,
                   param_distributions={'layrs': [4, 6, 8, 10, 12],
                                        'neurons': [5, 10, 15, 20, 25, 30, 35,
                                                    40, 45, 50, 55, 60]})

In [16]:
rnd_search_cv1.best_params_

{'neurons': 55, 'layrs': 6}

In [22]:
#grid2 used in GridSearchCV
grid2 = {
    'neurons' : [30, 40, 50, 60, 70, 80],
    'layrs' : [4, 6, 8, 10]
}

In [23]:
keras_reg2 = keras.wrappers.scikit_learn.KerasRegressor(model_development1)
grid_search_cv = GridSearchCV(keras_reg2, grid2, cv=3)

In [24]:
grid_search_cv.fit(Xtrain, Ytrain, epochs=100, validation_data=(Xval, Yval),
callbacks=[checkpoint_cb, early_stopping, lr_schedule])

C:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
2513/2513 [==============================] - 7s 2ms/step - loss: 470.4752 - coeff_determination: 0.9107 - val_loss: 128.6579 - val_coeff_determination: 0.9758
Epoch 2/100
2513/2513 [==============================] - 6s 3ms/step - loss: 118.5488 - coeff_determination: 0.9776 - val_loss: 112.6958 - val_coeff_determination: 0.9789
Epoch 3/100
2513/2513 [==============================] - 6s 2ms/step - loss: 109.7013 - coeff_determination: 0.9792 - val_loss: 105.8675 - val_coeff_determination: 0.9801
Epoch 4/100
2513/2513 [==============================] - 6s 3ms/step - loss: 106.3494 - coeff_determination: 0.9799 - val_loss: 105.3489 - val_coeff_determination: 0.9802
Epoch 5/100
2513/2513 [==============================] - 7s 3ms/step - loss: 104.7055 - coeff_determination: 0.9802 - val_loss: 105.8895 - val_coeff_determination: 0.9801
Epoch 6/100
2513/2513 [==============================] - 6s 2ms/step - loss: 103.3422 - coeff_determination: 0.9804 - val_loss: 102.6889 - val_co

GridSearchCV(cv=3,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x00000000143C0580>,
             param_grid={'layrs': [4, 6, 8, 10],
                         'neurons': [30, 40, 50, 60, 70, 80]})

In [38]:
grid_search_cv.best_params_

{'layrs': 8, 'neurons': 70}

In [47]:
# final model
final_model = keras.models.Sequential([
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal', input_shape=Xtrain.shape[1:]),
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(70, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dense(3)
])

In [49]:
final_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=0.001),metrics=[coeff_determination])

C:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [50]:
final_model.fit(Xtrain, Ytrain, epochs=100, validation_data=(Xval, Yval),
callbacks=[checkpoint_cb, early_stopping, lr_schedule])

Epoch 1/100
3769/3769 [==============================] - 13s 3ms/step - loss: 165.4637 - coeff_determination: 0.9687 - val_loss: 107.9986 - val_coeff_determination: 0.9797
Epoch 2/100
3769/3769 [==============================] - 13s 3ms/step - loss: 107.5097 - coeff_determination: 0.9796 - val_loss: 107.1405 - val_coeff_determination: 0.9798
Epoch 3/100
3769/3769 [==============================] - 12s 3ms/step - loss: 105.0073 - coeff_determination: 0.9801 - val_loss: 104.8251 - val_coeff_determination: 0.9803
Epoch 4/100
3769/3769 [==============================] - 12s 3ms/step - loss: 104.0173 - coeff_determination: 0.9803 - val_loss: 101.5936 - val_coeff_determination: 0.9809
Epoch 5/100
3769/3769 [==============================] - 13s 3ms/step - loss: 102.6486 - coeff_determination: 0.9805 - val_loss: 100.9467 - val_coeff_determination: 0.9810
Epoch 6/100
3769/3769 [==============================] - 13s 3ms/step - loss: 102.4591 - coeff_determination: 0.9806 - val_loss: 103.2442 - 

In [51]:
Ypredict = final_model.predict(Xtest)

In [56]:
r2_score(Ytest, Ypredict)

0.9448022121166019

In [64]:
print(Ypredict[21])
print(Ytest[21])

[128.28268   61.0968   -29.304314]
[128  82 -50]
